# BERT for sequence classification

In [3]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, \
     DataCollatorWithPadding, pipeline
from datasets import Dataset
import numpy as np
import evaluate

Downloading a training data with a sequence of classified entities

In [4]:
snips_file = open('../data/snips.train.txt', 'rb')

snips_rows = snips_file.readlines()

snips_rows[:20]

[b'listen O\r\n',
 b'to O\r\n',
 b'westbam B-artist\r\n',
 b'alumb O\r\n',
 b'allergic B-album\r\n',
 b'on O\r\n',
 b'google B-service\r\n',
 b'music I-service\r\n',
 b'PlayMusic\r\n',
 b'\r\n',
 b'add O\r\n',
 b'step B-entity_name\r\n',
 b'to I-entity_name\r\n',
 b'me I-entity_name\r\n',
 b'to O\r\n',
 b'the O\r\n',
 b'50 B-playlist\r\n',
 b'cl\xc3\xa1sicos I-playlist\r\n',
 b'playlist O\r\n',
 b'AddToPlaylist\r\n']

In [5]:
# This code segment parses the snips dataset into a more manageable format

utterances = [] # Stores the complete sentences
tokenized_utterances = [] # Stores the tokenized sentences
labels_for_tokens = [] # Stores the labels of each word in the sentence
sequence_labels = [] # Text

utterance, tokenized_utterance, label_for_utterances = '', [], []
for snip_row in snips_rows:
    if len(snip_row) == 2:  # skip over rows with no data
        continue
    if ' ' not in snip_row.decode():  # we've hit a sequence label
        sequence_labels.append(snip_row.decode().strip())
        utterances.append(utterance.strip())
        tokenized_utterances.append(tokenized_utterance)
        labels_for_tokens.append(label_for_utterances)
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue
    token, token_label = snip_row.decode().split(' ')
    token_label = token_label.strip()
    utterance += f'{token} '
    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)

In [6]:
len(labels_for_tokens), len(tokenized_utterances), len(utterances), len(sequence_labels)

(13084, 13084, 13084, 13084)

In [7]:
utterances[1], sequence_labels[1]

('add step to me to the 50 clásicos playlist', 'AddToPlaylist')

In [8]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

In [9]:
# Set of unique sequence labels
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels

['RateBook',
 'SearchCreativeWork',
 'GetWeather',
 'PlayMusic',
 'BookRestaurant',
 'AddToPlaylist',
 'SearchScreeningEvent']

In [10]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

print(f'There are {len(unique_sequence_labels)} unique sequence labels')

There are 7 unique sequence labels


In [11]:
from functools import reduce

unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))
labels_for_tokens = [[unique_token_labels.index(_) for _ in l] for l in labels_for_tokens]

print(f'There are {len(unique_token_labels)} unique token labels')

There are 72 unique token labels


In [12]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print([unique_token_labels[l] for l in labels_for_tokens[0]])
print(utterances[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[21, 21, 27, 21, 5, 21, 6, 60]
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
3
PlayMusic


In [13]:
Dataset

datasets.arrow_dataset.Dataset

In [14]:
snips_dataset = Dataset.from_dict(
    dict(
        utterance=utterances, 
        label=sequence_labels,
        tokens=tokenized_utterances,
        token_labels=labels_for_tokens
    )
)

snips_dataset = snips_dataset.train_test_split(test_size=0.2)

In [15]:
snips_dataset

DatasetDict({
    train: Dataset({
        features: ['utterance', 'label', 'tokens', 'token_labels'],
        num_rows: 10467
    })
    test: Dataset({
        features: ['utterance', 'label', 'tokens', 'token_labels'],
        num_rows: 2617
    })
})

In [16]:
unique_sequence_labels[6]

'SearchScreeningEvent'

In [17]:
snips_dataset['train'][0]

{'utterance': 'find a photograph called call on me',
 'label': 1,
 'tokens': ['find', 'a', 'photograph', 'called', 'call', 'on', 'me'],
 'token_labels': [21, 21, 68, 21, 30, 9, 9]}

In [18]:
tokenizer('hi')

{'input_ids': [101, 20844, 102], 'attention_mask': [1, 1, 1]}

In [19]:
tokenizer.decode([101, 2603, 1142, 18977, 126, 2940, 102])


'[CLS] rate this textbook 5 stars [SEP]'

In [20]:
# simple function to batch tokenize utterances with truncation
def preprocess_function(examples):
    return tokenizer(examples["utterance"], truncation=True)

In [21]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:  38%|███▊      | 4000/10467 [00:00<00:00, 11312.46 examples/s]

In [22]:
# only input_ids, attention_mask, and label are used. The rest are for show
seq_clf_tokenized_snips['train'][0]

{'utterance': 'find a photograph called call on me',
 'label': 1,
 'tokens': ['find', 'a', 'photograph', 'called', 'call', 'on', 'me'],
 'token_labels': [21, 21, 68, 21, 30, 9, 9],
 'input_ids': [101, 1525, 170, 10110, 1270, 1840, 1113, 1143, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
# DataCollatorWithPadding creates batch of data. It also dynamically pads text to the 
#  length of the longest element in the batch, making them all the same length. 
#  It's possible to pad your text in the tokenizer function with padding=True, dynamic padding is more efficient.

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
{i: l for i, l in enumerate(unique_sequence_labels)}

{0: 'BookRestaurant',
 1: 'SearchCreativeWork',
 2: 'PlayMusic',
 3: 'GetWeather',
 4: 'AddToPlaylist',
 5: 'SearchScreeningEvent',
 6: 'RateBook'}

In [23]:
sequence_clf_model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-cased', 
    num_labels=len(unique_sequence_labels),
)

# set an index -> label dictionary
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier

In [24]:
sequence_clf_model.config.id2label[0]

'BookRestaurant'

In [27]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):  # custom method to take in logits and calculate accuracy of the eval set
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    
    # some deep learning parameters that the Trainer is able to take in
    warmup_steps=len(seq_clf_tokenized_snips['train']) // 10,  # number of warmup steps for learning rate scheduler,
    weight_decay = 0.05,
    
    logging_steps=10,
    log_level='info',
    evaluation_strategy='epoch',
    eval_steps=50,
    save_strategy='epoch'
)

# Define the trainer:

trainer = Trainer(
    model=sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,  # optional
    data_collator=data_collator
)

In [27]:
# Get initial metrics
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens, utterance, token_labels. If tokens, utterance, token_labels are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 82/82 [01:14<00:00,  1.11it/s]


{'eval_loss': 1.9480133056640625,
 'eval_accuracy': 0.14214749713412303,
 'eval_runtime': 75.2707,
 'eval_samples_per_second': 34.768,
 'eval_steps_per_second': 1.089}

In [28]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens, utterance, token_labels. If tokens, utterance, token_labels are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/patrick/opt/anaconda3/envs/langflow/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 65,786,887
  2%|▏         | 10/656 [00:33<37:17,  

{'loss': 1.9525, 'learning_rate': 4.780114722753346e-07, 'epoch': 0.03}


  3%|▎         | 20/656 [01:03<31:01,  2.93s/it]

{'loss': 1.9461, 'learning_rate': 9.560229445506693e-07, 'epoch': 0.06}


  5%|▍         | 30/656 [01:33<31:29,  3.02s/it]

...

100%|██████████| 656/656 [33:58<00:00,  2.25s/it]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens, utterance, token_labels. If tokens, utterance, token_labels are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
                                                 
100%|██████████| 656/656 [35:13<00:00,  2.25s/it]Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json


{'eval_loss': 0.06147920712828636, 'eval_accuracy': 0.9858616736721437, 'eval_runtime': 74.586, 'eval_samples_per_second': 35.087, 'eval_steps_per_second': 1.099, 'epoch': 2.0}


Model weights saved in ./snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results/checkpoint-656 (score: 0.06147920712828636).
100%|██████████| 656/656 [35:16<00:00,  3.23s/it]

{'train_runtime': 2116.4004, 'train_samples_per_second': 9.891, 'train_steps_per_second': 0.31, 'train_loss': 0.512464398563635, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=0.512464398563635, metrics={'train_runtime': 2116.4004, 'train_samples_per_second': 9.891, 'train_steps_per_second': 0.31, 'train_loss': 0.512464398563635, 'epoch': 2.0})

In [29]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tokens, utterance, token_labels. If tokens, utterance, token_labels are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [01:12<00:00,  1.14it/s]


{'eval_loss': 0.06147920712828636,
 'eval_accuracy': 0.9858616736721437,
 'eval_runtime': 72.7994,
 'eval_samples_per_second': 35.948,
 'eval_steps_per_second': 1.126,
 'epoch': 2.0}

In [30]:
trainer.save_model()

Saving model checkpoint to ./snips_clf/results
Configuration saved in ./snips_clf/results/config.json
Model weights saved in ./snips_clf/results/pytorch_model.bin


In [31]:
# We can now load our fine-tuned from our directory
pipe = pipeline("text-classification", "./snips_clf/results", tokenizer=tokenizer)

pipe('Please add Here We Go by Dispatch to my road trip playlist')

loading configuration file ./snips_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./snips_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "BookRestaurant",
    "1": "SearchCreativeWork",
    "2": "PlayMusic",
    "3": "GetWeather",
    "4": "AddToPlaylist",
    "5": "SearchScreeningEvent",
    "6": "RateBook"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype":

[{'label': 'AddToPlaylist', 'score': 0.998089611530304}]

In [24]:
frozen_sequence_clf_model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-cased', 
    num_labels=len(unique_sequence_labels),
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.wei

In [25]:
# freezes EVERY parameter in our bert model
# does not freeze our classification layer
for param in frozen_sequence_clf_model.distilbert.parameters():
    param.requires_grad = False

In [30]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    
    # some deep learning parameters that the Trainer is able to take in
    warmup_steps=len(seq_clf_tokenized_snips['train']) // 10,  # number of warmup steps for learning rate scheduler,
    weight_decay = 0.05,
    report_to='wandb',
    
    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    eval_steps=50,
    save_strategy='epoch'
)

# Define the trainer:

trainer = Trainer(
    model=frozen_sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [31]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/patrick/opt/anaconda3/envs/langflow/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 595,975
Automatic Weights & Biases logging enabled

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: trendprix. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 1/656 [00:00<09:45,  1.12it/s]

...

                                                 
100%|██████████| 656/656 [12:18<00:00,  1.10it/s]Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json


{'eval_loss': 1.3543637990951538, 'eval_accuracy': 0.8914787925105082, 'eval_runtime': 71.9796, 'eval_samples_per_second': 36.358, 'eval_steps_per_second': 1.139, 'epoch': 2.0}


Model weights saved in ./snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results/checkpoint-656 (score: 1.3543637990951538).
100%|██████████| 656/656 [12:20<00:00,  1.13s/it]

{'train_runtime': 752.036, 'train_samples_per_second': 27.836, 'train_steps_per_second': 0.872, 'train_loss': 1.7816114474723979, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=1.7816114474723979, metrics={'train_runtime': 752.036, 'train_samples_per_second': 27.836, 'train_steps_per_second': 0.872, 'train_loss': 1.7816114474723979, 'epoch': 2.0})

In [32]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens. If token_labels, utterance, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [01:07<00:00,  1.21it/s]


{'eval_loss': 1.3543637990951538,
 'eval_accuracy': 0.8914787925105082,
 'eval_runtime': 68.4554,
 'eval_samples_per_second': 38.229,
 'eval_steps_per_second': 1.198,
 'epoch': 2.0}

In [34]:
# We can now load our fine-tuned from our directory
pipe = pipeline("text-classification", "./snips_clf/results", tokenizer=tokenizer)

loading configuration file ./snips_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./snips_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "BookRestaurant",
    "1": "SearchCreativeWork",
    "2": "PlayMusic",
    "3": "GetWeather",
    "4": "AddToPlaylist",
    "5": "SearchScreeningEvent",
    "6": "RateBook"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype":

In [38]:
pipe('I want to make a reservation at Clementina for tonight')

[{'label': 'BookRestaurant', 'score': 0.9962743520736694}]

wandb: Network error (ConnectionError), entering retry loop.
